In [14]:
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')
import pyspark
from pyspark.sql import *
import pandas as pd
from pyspark.sql.types import ArrayType,StringType,DoubleType,IntegerType
from pyspark.mllib.linalg import SparseVector, DenseVector
from pyspark.sql.functions import *

In [15]:
sc=pyspark.SparkContext()


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-2-a6e611c80d06>:1 

In [16]:
spark= SparkSession \
        .builder \
        .appName("PART 1") \
        .config("spark.some.config.option","15g") \
        .getOrCreate()

In [17]:

# df = spark.read.load("sample-movie-train.csv",format="csv", sep=",",inferSchema='true', header='true')

import pandas as pd
from pyspark.sql import SQLContext
p_df = pd.read_csv("train.csv")
# print(p_df)
sqlCtx = SQLContext(sc)
df = sqlCtx.createDataFrame(p_df)
df.show()
df.printSchema()

+--------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|
+--------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|
|  595909|   A Cry in the Dark|Seventh-day Adven...|['Crime Fiction',...|
| 5272176|            End Game|The president is ...|['Action/Adventur...|
| 1952976|          Dark Water|{{plot}} The film...|['Thriller', 'Dra...|
|24225279|                Sing|The story begins ...|           ['Drama']|
| 2462689|       Meet John Doe|Infuriated at bei...|['Black-and-white...|
|20532852|Destination Meatball|A line of people ...|['Animation', 'Sh...|
|15401493|    Husband for Hire|Lola  a

In [18]:
import pandas as pd
from pyspark.sql import SQLContext
test = pd.read_csv("test.csv")
# print(p_df)
sqlCtx = SQLContext(sc)
test = sqlCtx.createDataFrame(test)
test.printSchema()

root
 |-- movie_id: long (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)



In [19]:
df_map = spark.read.load("mapping.csv",format="csv", sep=",",inferSchema='true', header='true')
dd_map= df_map.select("0").collect()[:]

def map_gen(x):
    output_array=[]
    for gen in dd_map:
        if gen[0] in x:
            output_array.append(1)
        else:
            output_array.append(0)
    return output_array
             
mapper = udf(map_gen,ArrayType(IntegerType()))
genre_mapped = df.withColumn("mapped_genre", mapper("genre"))

In [20]:
genre_mapped.show()

+--------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|        mapped_genre|
+--------+--------------------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|[1, 0, 0, 0, 0, 1...|
|31186339|    The Hunger Games|The nation of Pan...|['Action/Adventur...|[1, 0, 0, 0, 1, 0...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|[1, 0, 0, 0, 1, 0...|
| 2231378|  The Lemon Drop Kid|The Lemon Drop Ki...|          ['Comedy']|[0, 1, 0, 0, 0, 0...|
|  595909|   A Cry in the Dark|Seventh-day Adven...|['Crime Fiction',...|[1, 0, 0, 0, 0, 1...|
| 5272176|            End Game|The president is ...|['Action/Adventur...|[1, 0, 0, 1, 1, 0...|
| 1952976|          Dark Water|{{plot}} The film...|['Thriller', 'Dra...|[1, 0, 0, 1, 0, 0...|
|24225279|                Sing|The story begins ..

In [21]:
num_label = len(dd_map)
result = genre_mapped.select(['movie_id', 'movie_name', 'plot', 'genre', 'mapped_genre']+[expr('mapped_genre[' + str(x) + ']') for x in range(0, num_label)])

new_colnames = ['movie_id', 'movie_name', 'plot', 'genre', 'mapped_genre'] + ['label_' + str(i) for i in range(0, num_label)] 
result = result.toDF(*new_colnames)
result.printSchema()

root
 |-- movie_id: long (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- mapped_genre: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- label_0: integer (nullable = true)
 |-- label_1: integer (nullable = true)
 |-- label_2: integer (nullable = true)
 |-- label_3: integer (nullable = true)
 |-- label_4: integer (nullable = true)
 |-- label_5: integer (nullable = true)
 |-- label_6: integer (nullable = true)
 |-- label_7: integer (nullable = true)
 |-- label_8: integer (nullable = true)
 |-- label_9: integer (nullable = true)
 |-- label_10: integer (nullable = true)
 |-- label_11: integer (nullable = true)
 |-- label_12: integer (nullable = true)
 |-- label_13: integer (nullable = true)
 |-- label_14: integer (nullable = true)
 |-- label_15: integer (nullable = true)
 |-- label_16: integer (nullable = true)
 |-- label_17: integer (nullable = true)
 |-- label_18: integer

In [22]:
from pyspark.ml.feature import HashingTF, Tokenizer, RegexTokenizer, StopWordsRemover, CountVectorizer, Word2Vec, PCA
from pyspark.ml import Pipeline

regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="words", pattern="\\W")
# remover = StopWordsRemover(inputCol="words", outputCol="filtered")
w2v = Word2Vec(inputCol="words", outputCol="features")
preprocessor = Pipeline(stages=[regexTokenizer, w2v ])
model = preprocessor.fit(result)
train = model.transform(result)

model = preprocessor.fit(test)
test = model.transform(test)

In [23]:
train.printSchema()
test.printSchema()


root
 |-- movie_id: long (nullable = true)
 |-- movie_name: string (nullable = true)
 |-- plot: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- mapped_genre: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- label_0: integer (nullable = true)
 |-- label_1: integer (nullable = true)
 |-- label_2: integer (nullable = true)
 |-- label_3: integer (nullable = true)
 |-- label_4: integer (nullable = true)
 |-- label_5: integer (nullable = true)
 |-- label_6: integer (nullable = true)
 |-- label_7: integer (nullable = true)
 |-- label_8: integer (nullable = true)
 |-- label_9: integer (nullable = true)
 |-- label_10: integer (nullable = true)
 |-- label_11: integer (nullable = true)
 |-- label_12: integer (nullable = true)
 |-- label_13: integer (nullable = true)
 |-- label_14: integer (nullable = true)
 |-- label_15: integer (nullable = true)
 |-- label_16: integer (nullable = true)
 |-- label_17: integer (nullable = true)
 |-- label_18: integer

In [25]:
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

# Model 1
output = []
models=[]
for i in range(num_label):
  label = "label_"+str(i)
  print(label)
  cur_train = train.select("movie_id", "movie_name", "features", label)
  lr = LogisticRegression(maxIter=1000,featuresCol="features", labelCol=label)
  model = lr.fit(cur_train)
  models.append(model)

  prediction = model.transform(test)

  # prediction.show(5)
  res = prediction.select('movie_id', 'prediction').collect()
  output.append(res)

label_0
label_1
label_2
label_3
label_4
label_5
label_6
label_7
label_8
label_9
label_10
label_11
label_12
label_13
label_14
label_15
label_16
label_17
label_18
label_19


In [26]:

from collections import defaultdict
op_dic = defaultdict(list)
for row in output:
  for i in row:
    op_dic[i["movie_id"]].append(i['prediction'])
 

In [27]:


import csv
with open('part3.csv', 'w') as f:
    f.write("%s,%s\n"%('movie_id', 'predictions'))
    for key in op_dic.keys():
        f.write("%s,%s\n"%(key, ' '.join(str(e)[0] for e in op_dic[key])))

In [31]:
num=0
for model in models:
  model.save("part3_genre_"+str(num)+"_model")
  num+=1

In [32]:
from pyspark.ml.classification import LogisticRegressionModel

num=0
out=[]
for i in range(num_label):
  lr= LogisticRegressionModel.load("part3_genre_"+str(num)+"_model")
  num+=1
  prediction = model.transform(test)
  res = prediction.select('movie_id', 'prediction').collect()
  out.append(res)

In [33]:

from collections import defaultdict
op_dic = defaultdict(list)
for row in output:
  for i in row:
    op_dic[i["movie_id"]].append(i['prediction'])
    


import csv
with open('part3.csv', 'w') as f:
    f.write("%s,%s\n"%('movie_id', 'predictions'))
    for key in op_dic.keys():
        f.write("%s,%s\n"%(key, ' '.join(str(e)[0] for e in op_dic[key])))
 